In [1]:
import datetime
from multiprocessing import Pool

import numpy as np
import pandas as pd

from test.test_baa import execute_experiment

In [2]:
use_parallel = True

In [3]:
num_qubits = 7
entanglement_bounds = (0.4, 1.0)
max_fidelity_loss = np.linspace(0.1, 1.0, 10)

data = [(i, num_qubits, entanglement_bounds, max_fidelity_loss, False, [1, 5]) for i in range(100)]
if use_parallel:
    with Pool() as pool:
        result = pool.starmap(execute_experiment, data)
else:
    result = [execute_experiment(*d) for d in data]

In [4]:
df = pd.concat(result, ignore_index=True)
df

,id,with_low_rank,strategy,num_qubits,depth,cnots,entganglement,entganglement (MW),max_fidelity_loss,total_saved_cnots,total_fidelity_loss,data,real_cnots,real_cnots_no_approx,real_depth,real_depth_no_approx,real_fidelity_loss,real_fidelity_loss_benchmark,duration
0,0,False,brute_force,7,14,151,0.571017,0.611636,0.1,96,0.054912,"[(0, [64]), (1, [2])]",54,151,89,364,0.0549,0.0,1.390601
1,0,False,greedy,7,14,151,0.571017,0.611636,0.1,96,0.054912,"[(0, [64]), (1, [2])]",54,151,89,364,0.0549,0.0,1.453099
2,0,True,brute_force,7,14,151,0.571017,0.611636,0.1,130,0.055367,"[(1, [2]), (2, [64])]",21,151,42,364,0.2203,0.0,1.375066
3,0,True,greedy,7,14,151,0.571017,0.611636,0.1,115,0.033907,"[(2, [128, 1])]",36,151,98,364,0.1951,0.0,1.749987
4,0,False,brute_force,7,14,151,0.571017,0.611636,0.2,146,0.190933,"[(0, [8]), (1, [2]), (0, [4]), (1, [2])]",5,151,17,364,0.1936,0.0,0.701098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,99,True,greedy,7,7,84,0.673096,0.628701,0.9,84,0.653879,"[(1, [2, 1]), (1, [2, 1]), (1, [2, 1]), (1, [2...",0,23,2,68,0.6731,-0.0,0.093751
3996,99,False,brute_force,7,7,84,0.673096,0.628701,1.0,84,0.653863,"[(1, [2, 1]), (1, [2, 1]), (1, [2, 1]), (1, [2...",0,23,2,68,0.6731,-0.0,0.100685
3997,99,False,greedy,7,7,84,0.673096,0.628701,1.0,84,0.673100,"[(1, [2, 1]), (1, [2, 1]), (1, [2, 1]), (1, [2...",0,23,2,68,0.6539,-0.0,0.109413
3998,99,True,brute_force,7,7,84,0.673096,0.628701,1.0,84,0.653889,"[(1, [2, 1]), (1, [2, 1]), (1, [2, 1]), (1, [2...",0,23,2,68,0.6731,-0.0,0.093708


In [5]:
# Tests:
# Calculation Tests
# The benchmark will not change the state at all, so it must be essentially zero
df['benchmark_fidelity_loss_pass'] = np.abs(df['real_fidelity_loss_benchmark']) < 1e-6
# The expected / predicted fidelity loss must be less or equal to the max fidelity loss
df['approximation_calculation_pass'] = df['max_fidelity_loss'] >= df['total_fidelity_loss']

# The real Tests
# The real measured fidelity measure must be less or equal to the configured mx fidelity loss
df['real_approximation_calculation_pass'] = df['real_fidelity_loss'] < df['max_fidelity_loss']
# The predicted maximum CNOT gates and the no-approximation count should be within 10%
df['cnot_prediction_calculation_pass'] = np.abs(df['real_cnots_no_approx'] - df['cnots']) < 0.1 * df['cnots']
# The predicted CNOT gates should be within an error margin of 10%
df['saved_cnots_calculation_pass'] = np.abs(df['cnots'] - df['total_saved_cnots'] - df['real_cnots']) <= 0.1 * (df['cnots'] - df['total_saved_cnots'])

In [6]:
timestamp_sec = int(datetime.datetime.now().timestamp())
df.to_pickle(f'./{timestamp_sec}.test_baa.pickle')
df.to_csv(f'./{timestamp_sec}.test_baa.csv')